In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import allel
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, cohen_kappa_score, f1_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline


import umap.umap_ as umap

In [4]:
R_SEED = 22
home = os.path.expanduser('~')
directory = os.path.join('Imp_Research','Dataset')

# populations = ['BFcol','BFgam','AOcol','CIcol','CMgam','FRgam',
#               'GAgam','GHcol','GHgam','GM','GNcol','GNgam','GQgam',
#               'GW','KE','UGgam']

# Using only 12 populations out of 16
populations = ['BFcol','BFgam','AOcol','CIcol','CMgam',
              'GAgam','GHcol','GM','GNgam','GW','KE','UGgam']

# Creating a dictionary of Population names with labels
populations_labels = {}
for i in range(len(populations)):
    populations_labels[i] = populations[i]

### Functions to Load and Pre-Process the Datasets

In [ ]:
'''
Requirements : NumPy and Scikit-Allel
'''

class FilterSNP():
    def __init__(self,haplotype, POS):
        self.haplotype = haplotype
        self.POS = POS
        self.H = haplotype
        self.P = POS
        self._removed_maf = None
        self._retained_maf = None
        self._retained_ld = None
        self._removed_ld = None
        self._unlinked_POS = None
        
    def all_filters(self, LD_window_size, LD_overlap_step, MBP_start = 1, MBP_end = 37, MAF_threshold = 5, LD_threshold=.1,LD_iter=1):
        print("1.) Selecting Mega Base Pairs")
        self.H,self.P = self.get_haplo_MBP(self.H,self.P,start = MBP_start, end = MBP_end)
        print("MBP selected. Retained Matrix = ", self.H.shape)
        print("2.) Filtering Rare Allels")
        self.H,self.P = self.filter_MAF(self.H,self.P,threshold = MAF_threshold)
        print("3.) Performing LD Pruning")
        self.H,self.P = self.LD_pruning(self.H, self.P, LD_window_size, LD_overlap_step, threshold = LD_threshold, n_iter = LD_iter)
        print("Retained Haplotype Matrix = ", self.H.shape)
        print("Retained Positions Matrix = ",self.P.shape)
        
        return self.H, self.P
        
    # input dimensions for Haplotype matrix (gn) = SNP X n
    def filter_MAF(self,haplo,POS,threshold = 5):
        if threshold >= 50 : 
            print("MAF threshold cannot be more than 49%")
            return
        samples = haplo.shape[1]
        sums = haplo.sum(axis=1)
        maf = self.get_MAF(haplo)
        #indexes = np.where(maf >= threshold*0.01)[0]
        minor = samples*threshold/100
        major = samples*(100-threshold)/100
        # Selects indexes where allels are >threshold or all 0 and all 1.
        indexes = np.where((sums>=minor)& (sums<=major))[0]
        print("Number of SNPs removed = ",len(haplo)-len(indexes))
        print("Retaining = ",len(indexes))
        self._removed_maf = len(haplo)-len(indexes)
        self._retained_maf = len(indexes)
        return np.take(haplo,indexes,0), np.take(POS,indexes,0)

    # Returns : Array of MAF
    # input dimensions for Haplotype matrix (gn) = SNP X n
    def get_MAF(self,haplo):
        samples = haplo.shape[1]
        sums = haplo.sum(axis=1)
        maf = []
        for s in sums:
            if s != samples or s != 0:
                frequency = s/samples
                if frequency > 0.5:
                    maf.append(1-frequency)
                else: 
                    maf.append(frequency)
        return np.array(maf)

    # input dimensions for Haplotype matrix (gn) = SNP X n
    def get_MBP(self,POS,start = 1,end = 37):
        return np.where(POS[np.where(POS>=1e6)]<=37e6)[0]

    # input dimensions for Haplotype matrix (gn) = SNP X n
    def get_haplo_MBP(self,haplotype,POS,start = 1,end = 37):
        index = self.get_MBP(POS,start,end)
        return np.take(haplotype,index,0),np.take(POS,index,0)
    
    # input dimensions for Haplotype matrix (gn) = SNP X n
    def LD_pruning(self, gn, pos, size, step, threshold = .1, n_iter=1):
        removed = 0
        for i in range(n_iter):
            
            # Returns a boolean array. True(1) - SNPs are within threshold (Unliked), False(0) = Linked
            loc_unlinked = allel.locate_unlinked(gn, size=size, step=step, threshold=threshold)
            
            # Counts non-zero elements = No. of unlinked loci or SNPs to retain
            n = np.count_nonzero(loc_unlinked)
            
            # Calculate the number of SNPs to be removed
            n_remove = gn.shape[0] - n
            removed += n_remove
            print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
            
            # Select only the unlinked SNPs, i.e., indexes where value is True(1).
            gn = gn.compress(loc_unlinked, axis=0)
            
            # retaining the indexes preserved
            pos = pos.compress(loc_unlinked)
        self._retained_ld = gn.shape[0]
        self._removed_ld = removed
        self._unlinked_POS = pos
        return gn,pos